In [12]:
import os
import glob
import pandas as pd
import numpy as np
import joblib
from tqdm.notebook import tqdm
bayes_results = glob.glob("Experiment results/Bayesian/Experiment1/*")
result1 = bayes_results[0]


['p_control=5', 'mde=0', 'prob_super=0.8']

In [128]:
from collections import Counter
def create_bayes_results(filepath):
    params = filepath.split("/")[-1].split(r"__")
    result_df = pd.DataFrame(data=list(map(lambda x: x.split("=")[1], params))
                      ).T
    result_df.columns = list(map(lambda x: x.split("=")[0], params))
    result_df.set_index(list(map(lambda x: x.split("=")[0], params)), inplace=True)
    result_df['count_winners'] = ''
    result_df['probability_superiority_mean'] = ''
    result_df['probability_superiority_median'] = ''
    result_df['expected_losses'] = ''
    result_df['size'] = ''
    df = joblib.load(filepath)
    winners = [df[i][0]['probability_superiority'] for i in range(1000)]
    winners = [i if i != "not_winner" else -1 for i in winners]
    result_df.at[result_df.index[0], "count_winners"] = Counter(winners)
    result_df.at[result_df.index[0], 'probability_superiority_mean'] = np.mean(
        [df[i][1]['probability_superiority'][0] for i in range(1000)], axis=0).round(3)
    result_df.at[result_df.index[0], 'probability_superiority_median'] = np.median(
        [df[i][1]['probability_superiority'][0]  for i in range(1000)], axis=0).round(3)
    result_df['expected_losses'] = df[i][1]['probability_superiority'][1].round(5)
    result_df['size'] = df[i][1]['probability_superiority'][2]
    return result_df
all_dfs = []
for result_file in glob.glob("Experiment results/Bayesian/Experiment1/*"):
    all_dfs.append(create_bayes_results(result_file))
all_dfs = pd.concat(all_dfs)
all_dfs

count_winners  \
p_control mde prob_super                              
5         0   0.8         {-1: 605, 1: 199, 0: 196}   
13        8   0.99                {1: 679, -1: 321}   
11        3   0.9                  {1: 914, -1: 86}   
3         2   0.9                  {1: 945, -1: 55}   
          6   0.95                {1: 869, -1: 131}   
...                                             ...   
11        8   0.99                {1: 670, -1: 330}   
          6   0.95                {1: 862, -1: 138}   
5         2   0.8                  {1: 981, -1: 19}   
3         4   0.8                  {1: 973, -1: 27}   
11        0   0.8         {-1: 611, 0: 194, 1: 195}   

                         probability_superiority_mean  \
p_control mde prob_super                                
5         0   0.8                      [0.499, 0.501]   
13        8   0.99                     [0.026, 0.974]   
11        3   0.9                        [0.03, 0.97]   
3         2   0.9                      [0.022, 0.978]   
          6   0.95                     [0.023, 0.977]   
...                                               ...   
11        8   0.99                     [0.027, 0.973]   
          6   0.95                     [0.027, 0.973]   
5         2   0.8                      [0.023, 0.977]   
3         4   0.8                      [0.025, 0.975]   
11        0   0.8                        [0.51, 0.49]   

                         probability_superiority_median  expected_losses  \
p_control mde prob_super                                                   
5         0   0.8                            [0.5, 0.5]          0.30038   
13        8   0.99                       [0.003, 0.997]          0.76818   
11        3   0.9                        [0.003, 0.997]          0.41855   
3         2   0.9                        [0.002, 0.998]          0.03555   
          6   0.95                       [0.003, 0.997]          0.20429   
...                                                 ...              ...   
11        8   0.99                       [0.003, 0.997]          0.74168   
          6   0.95                       [0.003, 0.997]          0.67378   
5         2   0.8                        [0.002, 0.998]          0.05088   
3         4   0.8                        [0.003, 0.997]          0.11042   
11        0   0.8                        [0.527, 0.473]          0.07381   

                             size  
p_control mde prob_super           
5         0   0.8           10000  
13        8   0.99          16965  
11        3   0.9          142968  
3         2   0.9         1281190  
          6   0.95         145080  
...                           ...  
11        8   0.99          20533  
          6   0.95          36200  
5         2   0.8          752699  
3         4   0.8          323365  
11        0   0.8           10000  

[270 rows x 5 columns]

In [129]:
all_dfs.sort_values("size")

count_winners  \
p_control mde prob_super                              
5         0   0.8         {-1: 605, 1: 199, 0: 196}   
9         0   0.8         {0: 194, -1: 624, 1: 182}   
3         0   0.99          {-1: 980, 0: 10, 1: 10}   
7         0   0.8         {-1: 603, 0: 201, 1: 196}   
5         0   0.85        {-1: 704, 0: 154, 1: 142}   
...                                             ...   
1         2   0.9                  {1: 937, -1: 63}   
              0.95                {1: 893, -1: 107}   
              0.99                {1: 721, -1: 279}   
              0.85                 {1: 959, -1: 41}   
              0.8                  {1: 974, -1: 26}   

                         probability_superiority_mean  \
p_control mde prob_super                                
5         0   0.8                      [0.499, 0.501]   
9         0   0.8                      [0.507, 0.493]   
3         0   0.99                     [0.494, 0.506]   
7         0   0.8                      [0.502, 0.498]   
5         0   0.85                     [0.499, 0.501]   
...                                               ...   
1         2   0.9                      [0.023, 0.977]   
              0.95                     [0.023, 0.977]   
              0.99                     [0.023, 0.977]   
              0.85                     [0.023, 0.977]   
              0.8                      [0.023, 0.977]   

                         probability_superiority_median  expected_losses  \
p_control mde prob_super                                                   
5         0   0.8                            [0.5, 0.5]          0.30038   
9         0   0.8                          [0.51, 0.49]          0.20296   
3         0   0.99                       [0.483, 0.517]          0.08074   
7         0   0.8                            [0.5, 0.5]          0.54763   
5         0   0.85                           [0.5, 0.5]          0.30038   
...                                                 ...              ...   
1         2   0.9                        [0.002, 0.998]          0.01489   
              0.95                       [0.002, 0.998]          0.01489   
              0.99                       [0.002, 0.998]          0.01489   
              0.85                       [0.002, 0.998]          0.01489   
              0.8                        [0.002, 0.998]          0.01489   

                             size  
p_control mde prob_super           
5         0   0.8           10000  
9         0   0.8           10000  
3         0   0.99          10000  
7         0   0.8           10000  
5         0   0.85          10000  
...                           ...  
1         2   0.9         3923647  
              0.95        3923647  
              0.99        3923647  
              0.85        3923647  
              0.8         3923647  

[270 rows x 5 columns]

In [130]:
len(glob.glob("/home/igor/Appbooster/proba.ai/AB/Experiment results/Thompson/Experiment1/*"))

159

In [89]:
result_df

,,,count_winners
p_control,mde,prob_super,
5,0,0.8,"{-1: 605, 1: 199, 0: 196}"


In [55]:
[df[i][0]['probability_superiority'] for i in range(1000)]

['not_winner',
 1,
 'not_winner',
 0,
 'not_winner',
 'not_winner',
 'not_winner',
 'not_winner',
 0,
 'not_winner',
 1,
 'not_winner',
 1,
 0,
 'not_winner',
 'not_winner',
 'not_winner',
 1,
 'not_winner',
 1,
 'not_winner',
 'not_winner',
 'not_winner',
 0,
 0,
 'not_winner',
 'not_winner',
 'not_winner',
 'not_winner',
 'not_winner',
 0,
 0,
 'not_winner',
 'not_winner',
 1,
 1,
 'not_winner',
 'not_winner',
 'not_winner',
 'not_winner',
 0,
 'not_winner',
 'not_winner',
 'not_winner',
 'not_winner',
 'not_winner',
 0,
 0,
 'not_winner',
 'not_winner',
 'not_winner',
 'not_winner',
 1,
 'not_winner',
 'not_winner',
 1,
 0,
 'not_winner',
 'not_winner',
 'not_winner',
 'not_winner',
 0,
 0,
 0,
 'not_winner',
 'not_winner',
 'not_winner',
 1,
 'not_winner',
 0,
 1,
 'not_winner',
 1,
 'not_winner',
 1,
 'not_winner',
 1,
 0,
 'not_winner',
 0,
 'not_winner',
 'not_winner',
 'not_winner',
 'not_winner',
 1,
 'not_winner',
 0,
 'not_winner',
 'not_winner',
 0,
 'not_winner',
 'not_win

In [ ]:
# Костыльное решение (фиксим, разделяя __)
p_control_list = [0.1, 0.2, 0.3, 0.4, 0.5]
mde_list = np.linspace(0.01, 0.5, 10)
# result_experiments_df = pd.DataFrame(index=pd.MultiIndex.from_product(
#                                      [p_control_list, mde_list],
#                                      names=["distr_type", "mean_control", "mde_test_effect",
#                                             "sigma_control", "sigma_diff", "size"]),
#                                      columns=['p_value_Student', 'p_value_MU', 'p_value_BS',
#                                               "control_bs_means", "test_bs_means"])

In [ ]:
import joblib
df = joblib.load("/home/igor/Appbooster/proba.ai/AB/Experiment results/Bayesian/Experiment1/p_control=1__mde=0__prob_super=0.8")

In [ ]:
df

In [ ]:
from tqdm.notebook import tqdm
results = []
for i in tqdm(range(10)):
    results.append(suka.start_experiment(seed=i))

In [ ]:
files = glob.glob('Experiment results/Bayesian/Experiment1/*')
for f in files:
    os.remove(f)

In [ ]:

n

In [ ]:
def get_size(theta_c, theta_t, alpha, beta):
    # вычисляем квантили нормального распределения
    t_alpha = stats.norm.ppf(1 - alpha, loc=0, scale=1)
    t_beta = stats.norm.ppf(beta, loc=0, scale=1)
    # решаем уравнение относительно n
    n = t_alpha*np.sqrt(theta_t*(1 - theta_t))
    n -= t_beta*np.sqrt(theta_c*(1 - theta_c))
    n /= theta_c - theta_t
    return int(np.ceil(n*n))

n_max = get_size(0.1, 0.11, 0.05, 0.2)
n_max